In [125]:
!pip install pathway bokeh panel --quiet

In [126]:
# Summer Analytics 2025 – Model 1 & 2
import pandas as pd
import numpy as np
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
from bokeh.io import output_notebook, show
output_notebook()

In [127]:
# Loading Dataset
df = pd.read_csv('/dataset.csv')

In [128]:
# Combining Date And Time
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

In [129]:
# Traffic to Numeric Scale
traffic_map = {'low': 1, 'average': 2, 'high': 3}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].astype(str).str.lower().map(traffic_map).fillna(0).astype(int)

In [130]:
# Vehicle Type to Numeric Scale
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleType'] = df['VehicleType'].str.lower().map(vehicle_weights).fillna(1.0)

In [131]:
# Selecting some columns for schema and new CSV
df = df[[
    "Timestamp", "SystemCodeNumber", "Occupancy", "Capacity",
    "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"
]]
df.to_csv("parking_stream.csv", index=False)

In [132]:
# Schema Types
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: float
    IsSpecialDay: int
    VehicleType: float

In [133]:
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Parsing Timestamp

fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    lot = data.SystemCodeNumber,
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [134]:
# MODEL 1

model_1 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_sum=pw.reducers.sum(pw.this.Occupancy),
        occ_count=pw.reducers.count(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
)

# Calculating Average Occupancy

model_1 = model_1_window.with_columns(
    occ_avg = pw.this.occ_sum / (pw.this.occ_count + 1e-5)
)

# Final Model 1 Price

model_1 = model_1.with_columns(
    price_model_1 = 10 + 5 * (pw.this.occ_avg / (pw.this.cap + 1e-5))
)

In [135]:
# MODEL 2

model_2 = (
    data_with_time.windowby(
        pw.this.t,
        instance=(pw.this.day, pw.this.lot),
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_sum = pw.reducers.sum(pw.this.Occupancy),
        occ_count = pw.reducers.count(pw.this.Occupancy),
        queue_sum = pw.reducers.sum(pw.this.QueueLength),
        traffic_sum = pw.reducers.sum(pw.this.TrafficConditionNearby),
        vehicle_sum = pw.reducers.sum(pw.this.VehicleType),
        record_count = pw.reducers.count(pw.this.QueueLength),
        event = pw.reducers.max(pw.this.IsSpecialDay),
        cap = pw.reducers.max(pw.this.Capacity)
    )
)

# Calculating Average Occupancy

model_2 = model_2.with_columns(
    occ_avg = pw.this.occ_sum / (pw.this.occ_count + 1e-5)
)

model_2 = model_2.with_columns(
    occ_avg = pw.this.occ_sum / (pw.this.occ_count + 1e-5),
    queue_sum = pw.this.queue_sum / (pw.this.record_count + 1e-5),
    traffic_sum = pw.this.traffic_sum / (pw.this.record_count + 1e-5),
    vehicle_sum = pw.this.vehicle_sum / (pw.this.record_count + 1e-5),
    demand_raw = (
        1.0 * (pw.this.occ_avg / (pw.this.cap + 1e-5)) +
        0.3 * pw.this.queue_sum -
        0.2 * pw.this.traffic_sum +
        2.0 * pw.this.event +
        0.5 * pw.this.vehicle_sum
    )
)

# Adding constant key for join
model_2 = model_2.with_columns(join_key=1)

demand_stats = model_2.reduce(
    min_demand = pw.reducers.min(pw.this.demand_raw),
    max_demand = pw.reducers.max(pw.this.demand_raw)
).with_columns(join_key=1)


# Joining Stats andd Final Price


model_2_A = model_2.join(
    demand_stats,
    pw.left.join_key == pw.right.join_key
).select(
    **pw.left,
    min_demand = pw.right.min_demand,
    max_demand = pw.right.max_demand
)

# Calculating Normalization
model_2_A = model_2_A.with_columns(
    demand_norm = (pw.this.demand_raw - pw.this.min_demand) /
                  (pw.this.max_demand - pw.this.min_demand + 1e-5)
)

# Final Model 2 price
model_2_A = model_2_A.with_columns(
    price_model_2 = 10 * (1 + 0.5 * pw.this.demand_norm)
)


In [136]:
df.to_csv('processed_parking_data.csv', index = False)

In [152]:
pn.extension()

def plot_model1(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 1 Pricing (Linear based on Occupancy)",
        x_axis_type="datetime"
    )
    #fig.line("t", "price_m1", source=source, color="blue", legend_label="Model 1")
    fig.line("t", "price_model_1", source=source, line_width=2, color="navy", legend_label="Model 1")
    fig.circle("t", "price_model_1", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

viz = model_1.plot(plot_model1, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [153]:
# STEP 6: Run the Pathway Stream (Activate at the End)
# ---------------------------------------------
# Uncomment the following to run in notebook
pw.run()


Output()

In [148]:
pn.extension()

def plot_model2(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Model 2 Pricing",
        x_axis_type="datetime"
    )
    #fig.line("t", "price_m2", source=source, color="blue", legend_label="Model 1")
    fig.line("t", "price_model_2", source=source, line_width=2, color="blue", legend_label="Model 2")
    fig.circle("t", "price_model_2", source=source, size=6, color="red")
    fig.legend.location = "top_left"
    return fig

viz1 = model_2_A.plot(plot_model2, sorting_col="t")
pn.Column(viz1).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [149]:
# STEP 6: Run the Pathway Stream (Activate at the End)
# ---------------------------------------------
# Uncomment the following to run in notebook
pw.run()


PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            21                                     
    PythonRead…            0         18368        18368    output           87         66                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…            8         15783        15783     Above you can see the latency of input and output      
    PythonRead…            8         15461        15461    operators. The latency is measured as the difference    
    PythonRead…            8         15314        15314   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            20                                     
    PythonRead…            0         18368        18368    output           74         54                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…            7         16335        16335     Above you can see the latency of input and output      
    PythonRead…            7         16001        16001    operators. The latency is measured as the difference    
    PythonRead…            7         15869        15869   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            51                                     
    PythonRead…            0         18368        18368    output          169        118                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…           16         16990        16990     Above you can see the latency of input and output      
    PythonRead…           16         16649        16649    operators. The latency is measured as the difference    
    PythonRead…           16         16529        16529   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            57                                     
    PythonRead…            0         18368        18368    output          211        154                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…           16         17101        17101     Above you can see the latency of input and output      
    PythonRead…           16         16755        16755    operators. The latency is measured as the difference    
    PythonRead…           15         16636        16636   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input            45                                     
    PythonRead…            0         18368        18368    output          161        116                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…           13         17345        17345     Above you can see the latency of input and output      
    PythonRead…           13         16997        16997    operators. The latency is measured as the difference    
    PythonRead…           13         16888        16888   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

...

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           197                                     
    PythonRead…     finished         18368        18368    output          311        114                          
    PythonRead…            0         18368        18368                                                            
    PythonRead…           14         17753        17753     Above you can see the latency of input and output      
    PythonRead…           14         17406        17406    operators. The latency is measured as the difference    
    PythonRead…           14         17296        17296   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           336                                     
    PythonRead…     finished         18368        18368    output          804        468                          
    PythonRead…     finished         18368        18368                                                            
    PythonRead…           76         18047        18047     Above you can see the latency of input and output      
    PythonRead…           75         17699        17699    operators. The latency is measured as the difference    
    PythonRead…           76         17590        17590   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input           163                                     
    PythonRead…     finished         18368        18368    output         1365       1202                          
    PythonRead…     finished         18368        18368                                                            
    PythonRead…     finished         18223        18223     Above you can see the latency of input and output      
    PythonRead…     finished         18340        18340    operators. The latency is measured as the difference    
    PythonRead…           39         18230        18230   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished         18368        18368    output          840       None                          
    PythonRead…     finished         18368        18368                                                            
    PythonRead…     finished         18223        18223     Above you can see the latency of input and output      
    PythonRead…     finished         18340        18340    operators. The latency is measured as the difference    
    PythonRead…     finished         18343        18343   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    

PATHWAY PROGRESS DASHBOARD                                             
                                                                                                                   
                         no.                                           latency                                     
                    messages                                           to wall     lag to                          
                      in the                                             clock      input      total    current    
                        last   in the last        since    operator       [ms]       [ms]       rows       rows    
    connector      minibatch        minute        start   ──────────────────────────────────────────────────────   
   ─────────────────────────────────────────────────────   input      finished                                     
    PythonRead…     finished         18368        18368    output          786       None                          
    PythonRead…     finished         18368        18368                                                            
    PythonRead…     finished         18223        18223     Above you can see the latency of input and output      
    PythonRead…     finished         18340        18340    operators. The latency is measured as the difference    
    PythonRead…     finished         18343        18343   between the time when the operator processed the data    
                                                               and the time when pathway acquired the data.        
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                                   
                                                                                                    